In [1]:
import os
import PIL
import cv2
import scipy
import pathlib
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)
import tensorflow as tf
import PIL.Image as Image
import tqdm as notebook_tqdm
import tensorflow_datasets as tfds
from matplotlib import pyplot as plt
from sklearn.preprocessing import OneHotEncoder

In [69]:
dataset_folder = "..\\archive"  
no_defect_images_folder = os.path.join(dataset_folder, "NODefect_images")
defect_images_folder = os.path.join(dataset_folder, "Defect_images")
mask_images_folder = os.path.join(dataset_folder, "Mask_images")

def gather_filenames(mypath):
    filepaths = []
    for path, _, files in os.walk(mypath):
        for name in files:
            filepaths.append(os.path.join(path, name))
    return filepaths

defect_file_paths = gather_filenames(defect_images_folder)
no_defect_file_paths = gather_filenames(no_defect_images_folder)


## Dataset
The textile fabric database consists of 245 images of 7 different fabrics. There are 140 defect-free images, 20 for each type of fabric. With different types of defects, there are 105 images.

Images have a size of 4096×256 pixels. Defective images have been denominated as follows: nnnn_ddd_ff.png, where nnnn is the image number, ddd is the defect code, and ff is the fabric code.

In [100]:
df = pd.DataFrame(defect_file_paths + no_defect_file_paths, columns=['filepath'])
df['id'] = df['filepath'].map(lambda x: int(x.split('\\')[-1].split('_')[0]))
df['defect'] = df['filepath'].map(lambda x: int(x.split('\\')[-1].split('_')[1]))
df['fabric'] = df['filepath'].map(lambda x: int(x.split('\\')[-1].split('_')[2].split('.')[0]))


In [101]:
# Defect codes
DEFECT_CODES = {
    0:  'No defect',
    2:	'Broken end',
    6:	'Broken yarn',
    10:	'Broken pick',
    16:	'Weft curling',
    19:	'Fuzzyball',
    22:	'Cut selvage',
    23:	'Crease',
    25:	'Warp ball',
    27:	'Knots',
    29:	'Contamination',
    30:	'Nep',
    36:	'Weft crack'
}

# Create encoding to be 1-13 instead of 2-36. This is the model output.
DEFECT_ENCODINGS = {}

for i, key in zip(range(len(DEFECT_CODES)) , DEFECT_CODES.keys()):
    DEFECT_ENCODINGS[key] = i

df['defect'] = df['defect'].map(lambda x: DEFECT_ENCODINGS[x])

In [107]:
x = df['filepath'].to_numpy() 

enc = OneHotEncoder()
enc.fit([[i] for i in df['defect']]) 

y = enc.transform([[i] for i in df['defect']]).toarray()

In [110]:
file_ds = tf.data.Dataset.from_tensor_slices((x, y))

In [112]:
def process_img(file_path, label):

    img = tf.io.read_file(file_path)
    img = tf.io.decode_png(img, channels=1)
    
    return img, label

image_ds = file_ds.map(process_img)

Feature Engineering

Supervised Learning

Unsupervised or Semisupervised Learning

Model Deployment/Evaluation